In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'modulo-11-suport-material:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5323227%2F8844367%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240726%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240726T172752Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1335e4fec9334589333cffdcc62cd5e9ac0d9f1a3b2b0ceeff603a2bc0734e35d3141c2ee257efb386e26a4c2565725bd8f1d1ee7c8fcefece781e3039d1651d44324a1ade93cc12744c1407cc3b03d4642d82b712a196956a862923854b50d1ca3b1c857979a918af83796d15a73c17dbfb2c3da1b986aee725bc6fa4213fed410f80c094d1290fd2f12274f82b5d54ff165117dd754256814ff7b2e0b069d71cfd0e024ec294725d20791026dc032a8a5728f82e1089f7ee88a36336fcfe5948dac03c6f026306ab54a6ea75e5f94606ecc51f421d81bbf5379e29a64cfc474f221c7eab93b0e8b40250226d0443fadab20a0c4b2035166022d77eb3cf8686'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

df = pd.read_csv('/kaggle/input/modulo-11-suport-material/Python_M10_support material (1).csv', na_values='na')

qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default']==0].shape
qtd_inadimplentes, _ = df[df['default']==1].shape
print(qtd_adimplentes+qtd_inadimplentes)

# Podemos notar que nosso arquivo é coerente com relação ao default, pois a soma dos
# defaults iguais a 0 com os defaults iguais a 1, possui cardinalidade igual ao total
# dos dados. Ou seja: não existe nenhum dado vazio dentro da coluna "default", e, portanto,
# não existe nenhum cliente que não esteja contabilizado nem como adimplente nem como
# adimplente, o que representaria uma incoerência dos dados.

In [ ]:
### Em termos de porcentagem, temos que:
print(f'O percentual de clientes adimplentes é: {qtd_adimplentes/qtd_total*100:.2f}%')
print(f'O percentual de clientes inadimplentes é: {qtd_inadimplentes/qtd_total*100:.2f}%')

### Este trecho do código mostra que o percentual de clientes adimplentes supera em muito o
### Percentual de clientes inadimplentes, o que também era o esperado

In [ ]:
print(df.dtypes)
print(df.select_dtypes('object').describe().transpose())

# Neste trecho, podemos ver algumas peculiaridades dos dados. Aparecem no segundo print
# Algumas coisas já esperadas, e outras nem tanto:
# O gênero mais frequente é o feminino. O que era esperado, visto que o gênero feminino
# é maioria na pirâmide etária

# A escolaridade mais frequente é o mestrado, o que já não parece tão comum, pelo menos em
# um país com baixa escolaridade média, como o Brasil, onde apenas cerca de 0,84% da
# população possui mestrado. Este aspecto dos dados talvez possa indicar que eles foram
# extraídos da população de um país mais desenvolvido...

# O salário mais comum é abaixo de 40k anuais, o que é totalmente esperado.

# O cartão blue ser o mais comum, igualmenete, visto que é o mais acessível...

# Os outros dois dados (limite de crédito e valor das transações), não possuem tanto
# significado quanto à análise de incidência de valor específico.
# O modo correto de se analisar seria via intervalos.

In [ ]:
print(df.drop('id', axis = 1).select_dtypes('number').describe().transpose())

# Nesta secção dos dados, vemos algo que parece corroborar com uma de nossas conjecturas
# anteriores. A média das idades é de 46, sinal de uma população bastante envelhecida
# bastante compatível com a hipótese de os dados serem oriundos da população de um país
# mais desenvolvido. A media de dependetes, no entanto, esperaria-se talvez que fosse menor

In [ ]:
df.isna().any()

# Os nossos dados faltantes estão presentes apenas nas colunas "escolaridade", "estado_civil"
# e salário anual. Estes dados faltantes possivelmente são equivalentes à "sem informação"
# ou até mesmo algum modo de representar o vazio, ou o zero. Portanto, provavelmente não
# representa um grande problema.

# Vamos extrair algumas informações a respeito dos dados faltantes á seguir:

In [ ]:
def status_dados_faltantes(df: pd.DataFrame) -> None:


    stats_dados_faltantes = []
    for col in df.columns:
        if df[col].isna().any():
            qtd, _ = df[df[col].isna()].shape
            total, _ = df.shape
            dict_dados_faltantes = {col:{'quantidade': qtd, 'porcentagem': round(qtd/total*100, 2)}}
            stats_dados_faltantes.append(dict_dados_faltantes)
    for stats in stats_dados_faltantes:
        print(stats)


In [ ]:
status_dados_faltantes(df=df)

# Com base nos resultados, podemos dizer que a quantidade de dados faltantes não são altas
# o suficiente para que possamos dizer que a análise dos dados está comprometida. Até porque:
print('\n')
status_dados_faltantes(df=df[df['default'] == 0])
print('\n')
status_dados_faltantes(df=df[df['default'] == 1])

# Podemos ver que o percentual relativo entre os clientes adimplentes que possuem dados
# faltando e o percentual dos clientes inadimplentes que possuem dados faltando
# é aproximadamente o mesmo. Ou seja, a faltância dos dados não afeta significativamente
# o default.


In [ ]:
limpeza_dos_dados = lambda valor: float(valor.replace('.','').replace(',','.'))

# Um dos problemas dos nossos dados é o fato de os valores das colunas "valor_trasacoes_12m"
# e "limite_credito" estarem sendo tratados como objetos, quando seria muito mais simples
# tratá-las como floats. Este erro acontece pois o pandas opera utilizando o sistema
# norte-americano, e os dados estão no modelo brasileiro. Para tratar isso, definimos
# a função lambda limpeza_dos_dados, e aplicaremos ela na base de dados utilizando o
# método apply.

df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(limpeza_dos_dados)
df['limite_credito'] = df['limite_credito'].apply(limpeza_dos_dados)

# Para checar se funcionou, observamos novamente o schema:

print(df.dtypes)

# Podemos ver, agora, o limite e o valor das transações dos últimos 12 meses corretamente
# descritos como floats.

In [ ]:
# Vamos descrever novamente os dados...

print(df.dtypes)
print('\n')
print(df.select_dtypes('object').describe().transpose())
print('\n')
print(df.drop('id', axis=1).select_dtypes('number').describe().transpose())

In [ ]:
# Vamos usar o pandas para remover os dados faltantes, de modo que eles não sejam
# levados em consideração em  nossas análises à partir de agora.

df.dropna(inplace=True)

# Agora re-analisamos os dados:

df.shape

# Uma porção significativa de nosso estoque de dados foi removida, mas não o suficiente
# para ser comprometedora, especialmente levando em consideração que não havia relação entre
# o default e a vacância de dados

# Agora, analisaremos os dados desconsiderando os clientes com vacancia de dados.

qtd_total_clean, _ = df.shape
adimplentes_clean, _ = df[df['default']==0].shape
inadimplentes_clean, _ = df[df['default']==1].shape

print(f'A quantidade de clientes adimplentes era {qtd_adimplentes}, e passou a ser {adimplentes_clean}')
print(f"""A proporção de clientes adimplentes anteriormente era
{qtd_adimplentes/qtd_total*100:.2f}%, e passou a ser {adimplentes_clean/qtd_total_clean*100:.2f}%\n""")

print(f'A quantidade total de clientes inadimplentes era {qtd_inadimplentes} e passou a ser {adimplentes_clean}')
print(f"""A proporção de clientes inadimplentes era {qtd_inadimplentes/qtd_total*100:.2f}%
e passou a ser {inadimplentes_clean/qtd_total_clean*100:.2f}%""")

In [ ]:
df_adimplente = df[df['default']==0]
df_inadimplente = df[df['default']==1]

# De agora em diante, faremos a exploração gráfica dos dados, agora já devidamente limpos
# E prontos para processamento
df_adimplente = df[df['default'] == 0]
df_inadimplente = df[df['default'] == 1]

# Títulos para os gráficos
coluna = 'escolaridade'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes',
           'Escolaridade dos Clientes Inadimplentes']
max_y = 0

# Configuração do tamanho dos gráficos
figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharex=True)

# Plotando os gráficos
for dataframe, eixo in zip([df, df_adimplente, df_inadimplente], range(3)):
    df_to_plot = dataframe[coluna].value_counts().reset_index()
    df_to_plot.columns = [coluna, 'frequencia_absoluta']
    df_to_plot.sort_values(by=[coluna], inplace=True)

    f = sns.barplot(x=coluna, y='frequencia_absoluta', data=df_to_plot, ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))

figura.tight_layout()
plt.show()


# Ao que podemos ver, a estrutura dos dados permanece a mesma com relação à escolaridade.
# Isto é, não parece haver uma tendência em perticular de um cliente ser ou não ser
# Inadimplente com base em sua escolaridade.

In [ ]:
# Agora, procuraremos verificar uma relação entre o salário anual e o default.
coluna = 'salario_anual'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes',
           'Escolaridade dos Clientes Inadimplente']
max_y = 0

figura, eixos = plt.subplots(1, 3, figsize=(20,5), sharex=True)

for dataframe, eixo in zip([df,df_adimplente,df_inadimplente], range(3)):
    df_to_plot = dataframe[coluna].value_counts().reset_index()
    df_to_plot.columns = [coluna, 'frequencia absoluta']
    df_to_plot.sort_values(by=[coluna], inplace=True)

    f = sns.barplot(x=coluna, y= 'frequencia absoluta', data=df_to_plot, ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequencia Absoluta')
    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0,max_y))

figura.tight_layout()
plt.show

In [ ]:
# O gráfico anterior também não explica o default, pois a proporção se mantém do mesmo modo
# Que antes. Uma pessoa ter um salário anual maior ou menor, não parece afetar
# Significativamente o default.

# Analisemos, então, outro setor do gráfico...

coluna = 'qtd_transacoes_12m'
titulos = ['Qtd de transacoes do ultimo ano',
           'Qtd de transacoes do ultimo ano de adimplentes',
           'Qtd transacoes do ultimo ano de inadimplentes']

max_y = eixo = 0
figora, eixos = plt.subplots(1, 3, figsize= (20,5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:
    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequencia Absoluta')

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim = (0,max_y))
    eixo += 1

figura.tight_layout()
plt.show

# Podemos, finalmente, reparar algumas alterações! Caso a quantidade de transações dos
# Últimos 12 meses não afetasse o default, esperariamos que o formato do gráfico se
# Mantivesse tanto dentro do geral, quanto dentro do default 0 ou default 1. Não é o que
# Ocorre. Podemos notar que a região do valor modal dos clientes ADIMPLENTES é totalmente
# Diferente da região modal dos clientes INADIMPLENTES. Ou seja, os gráficos sugerem que
# Existe uma clara TENDÊNCIA entre pessoas com em torno de 45 transações no útimo ano serem
# Clientes em default = 1, ou seja, inadimplentes. Em compensação, clientes com mais de 80
# Transações nos últimos 12 meses, parecem ter uma chance quase que nula de se tornarem
# Inadimplentes. Ou seja, este parâmetro parece VALIOSO para explicar o que leva o cliente
# A entrar em default.

In [ ]:
# Procuraremos agora uma relação entre o default e o valor das transações dos últimos 12
# Meses, utilizando, assim como anteriormente, um gráfico de histogramas, visto que os
# Valores estão livres, e não em intervalos bem definidos.

coluna = 'valor_transacoes_12m'
titulos = ['Qtd. de transacoes do ultimo ano',
          'Qtd. de transacoes do ultimo ano de adimplentes',
          'Qtd. de transacoes do ultimo ano de inadimplentes']

max_y = eixo = 0
figura, eixos = plt.subplots(1, 3, figsize=(20,5), sharex=True)
for dataframe in [df, df_adimplente, df_inadimplente]:
    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequencia Absoluta')

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))
    eixo += 1

figura.tight_layout()
plt.show



In [ ]:
# Agora vamos plotar ambos no mesmo gráfico para ver se ainda conseguimos estabelecer mais
# alguma relação. Vejamos:
f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m',
                    data=df, hue='default')
_ = f.set(
    title='Relação entre valor e quantidade de transações no último ano',
    xlabel='Valor das transações do último ano',
    ylabel='Quantidade de transacoes do ultimo ano')

# Neste gráfico, podemos reparar a relação entre as duas variáveis que foram capazes de
# explicar o default. Percebemos que existe uma enorme tendência de default entre pessoas
# que: ou transferem anualmente cerca de 7500 a 10000 reais em cerca de 60 a 80 transações
# ou transferem menos de 2500 distribuídas entre 0 e 60 transações. Existe também uma
# tendencia menor de default entre clientes que transacionam entre 5000 e 7500 entre 40
# e 60 transações. Outra questão que é valiosa, é a tendência basicamente absoluta, ao
# que podemos observar, de que clientes que transacionam mais de 12500 de não cometerem
# ato de inadimplência, o que é uma informação valiosa quanto à seguridade de empréstimo
# e conferimento de limite a tais clientes.

In [ ]:
    """SÍNTESE:
    EM NOSSA BUSTA POR EXPLICAR O DEFAULT, PROCURAMOS POR TENDÊNCIAS DE INADIMPLENCIA
    INICIALMENTE COM RELAÇÃO AO SALÁRIO ANUAL, E ESCOLARIDADE. NO ENTANDO, NOTAMOS QUE TAL
    TENDÊNCIA NÃO SE EVIDENCIA. ISTO POIS, COMO NOTAMOS, OS GRÁFICOS QUE MOSTRAVAM A
    ESCOLARIDADE E SALÁRIO ANUAL GERAL, E OS GRÁFICOS QUE MOSTRAM A ESCOLARIDADE E SALÁRIO
    ANUAL APENAS DOS CLIENTES ADIMPLENTES OU INADIMPLENTES SÃO O QUE, MATEMATICAMENTE,
    CHAMAMOS DE TRANSFORMAÇÕES LINEARES UM DO OUTRO (APROXIMADAMENTE). EM LINGUAGEM CASUAL,
    DIZEMOS QUE OS GRÁFICOS SÃO UMA VERSÃO 'ESPICHADA' UNS DOS OUTROS. OU TAMBÉM, DE UM
    MODO UM POUCO MAIS PRECISO, MANTÉM A PROPORCIONALIDADE (OU SEJA, A QUANDIDADE DE
    CLIENTES INADIMPLENTES COM CADA ESCOLARIDADE SÓ É MENOR QUE O GRÁFICO GERAL AO MESMO
    PASSO EM QUE EXISTEM MENOS CLIENTES INADIMPLENTES DO QUE CLIENTES GERAIS (OBVIAMENTE))"""
